# Janggu integrates with the tensorflow.data pipeline

In this notebook we illustrate how to exploit tensorflow.data pipelines with genomics datasets from janggu.

We shall use the familiar toy example about distinguishing Oct4 and Mafk binding sites based on the DNA sequence.

In [1]:
import os

import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input

from pkg_resources import resource_filename

from janggu.data import Bioseq
from janggu.data import Cover
from janggu.data import ReduceDim
from janggu.layers import DnaConv2D

from sklearn.metrics import roc_auc_score

In [2]:
from IPython.display import Image

np.random.seed(1234)

First, we need to specify the output directory in which the results are stored and define the required files:

In [3]:
os.environ['JANGGU_OUTPUT'] = '/home/wkopp/janggu_examples'

order = 3

# load the dataset
# The pseudo genome represents just a concatenation of all sequences
# in sample.fa and sample2.fa. Therefore, the results should be almost
# identically to the models obtained from classify_fasta.py.
REFGENOME = resource_filename('janggu', 'resources/pseudo_genome.fa')
# ROI contains regions spanning positive and negative examples
ROI_TRAIN_FILE = resource_filename('janggu', 'resources/roi_train.bed')
ROI_TEST_FILE = resource_filename('janggu', 'resources/roi_test.bed')
# PEAK_FILE only contains positive examples
PEAK_FILE = resource_filename('janggu', 'resources/scores.bed')

Next load the dataset from the files into the Bioseq and Cover objects. We shall load the training and test set separately.

In [4]:
# Training input and labels are purely defined genomic coordinates
DNA = Bioseq.create_from_refgenome('dna', refgenome=REFGENOME,
                                   roi=ROI_TRAIN_FILE,
                                   binsize=200,
                                   order=order,
                                   cache=True)


LABELS =Cover.create_from_bed('peaks', roi=ROI_TRAIN_FILE,
                               bedfiles=PEAK_FILE,
                               binsize=200,
                               resolution=200,
                               cache=True,
                               storage='sparse')


DNA_TEST = Bioseq.create_from_refgenome('dna', refgenome=REFGENOME,
                                        roi=ROI_TEST_FILE,
                                        binsize=200,
                                        order=order)

LABELS_TEST = Cover.create_from_bed('peaks',
                                    bedfiles=PEAK_FILE,
                                    roi=ROI_TEST_FILE,
                                    binsize=200,
                                    resolution=200,
                                    storage='sparse')


In [5]:
print('training set:', DNA.shape, LABELS.shape)
print('test set:', DNA_TEST.shape, LABELS_TEST.shape)

training set: (7797, 198, 1, 64) (7797, 1, 1, 1)
test set: (200, 198, 1, 64) (200, 1, 1, 1)


We obtain 7797 training sequences and 200 test sequences, each of length 200 bases. 
Because we use higher-order sequence representation, the sequence length is shrunken slightly.

## Prepare training tf.data pipeline

Next, we make use the `to_tf_dataset` method that is exposed by the Bioseq and Cover class. 
This method converts the respective genomic datasets into `tensorflow.data.Dataset` components.
Subsequently, one can take advantage of the utilities offered by tensorflow to further convert or reshape the tensors. Additional specialized methods for data augentation in genomics are briefly summarized at the end of this tutorial.

In [6]:
dna_ds = DNA.to_tf_dataset().map(tf.sparse.to_dense)

In [7]:
label_ds = LABELS.to_tf_dataset(as_sparse=False).map(lambda x: tf.reduce_sum(x, axis=(0,1)))

In [8]:
ds=tf.data.Dataset.zip((dna_ds, label_ds)).shuffle(buffer_size=10000).batch(32)

## Define and fit the network with the data pipeline

Specify a keras model with compatible dimesions input and output dimensions for the example.

In [9]:
xin = Input((200 - order + 1, 1, pow(4, order)))
layer = DnaConv2D(Conv2D(30, (21, 1),
                         activation='relu'))(xin)
layer = GlobalAveragePooling2D()(layer)
output = Dense(1, activation='sigmoid')(layer)



model = Model(xin, output)

model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['acc'])
model.summary()



Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 198, 1, 64)]      0         
_________________________________________________________________
dna_conv2d (DnaConv2D)       (None, 178, 1, 30)        40350     
_________________________________________________________________
global_average_pooling2d (Gl (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 31        
Total params: 40,381
Trainable params: 40,381
Non-trainable params: 0
_________________________________________________________________


In [10]:
hist = model.fit(ds, epochs=100)

print('#' * 40)
print('loss: {}, acc: {}'.format(hist.history['loss'][-1],
                                 hist.history['acc'][-1]))
print('#' * 40)

Epoch 1/100
244/244 [==============================] - 6s 23ms/step - loss: 0.5603 - acc: 0.7285
Epoch 2/100
244/244 [==============================] - 7s 28ms/step - loss: 0.4683 - acc: 0.7895
Epoch 3/100
244/244 [==============================] - 6s 24ms/step - loss: 0.4459 - acc: 0.7965
Epoch 4/100
244/244 [==============================] - 6s 24ms/step - loss: 0.4349 - acc: 0.8018
Epoch 5/100
244/244 [==============================] - 6s 23ms/step - loss: 0.4303 - acc: 0.8045
Epoch 6/100
244/244 [==============================] - 5s 20ms/step - loss: 0.4216 - acc: 0.8075
Epoch 7/100
244/244 [==============================] - 5s 21ms/step - loss: 0.4166 - acc: 0.8133
Epoch 8/100
244/244 [==============================] - 5s 20ms/step - loss: 0.4088 - acc: 0.8151
Epoch 9/100
244/244 [==============================] - 5s 19ms/step - loss: 0.4021 - acc: 0.8201
Epoch 10/100
244/244 [==============================] - 5s 20ms/step - loss: 0.3936 - acc: 0.8258
Epoch 11/100
244/244 [=======

244/244 [==============================] - 5s 18ms/step - loss: 0.0168 - acc: 0.9994
Epoch 85/100
244/244 [==============================] - 5s 19ms/step - loss: 0.0162 - acc: 0.9995
Epoch 86/100
244/244 [==============================] - 5s 19ms/step - loss: 0.0155 - acc: 0.9994
Epoch 87/100
244/244 [==============================] - 5s 19ms/step - loss: 0.0146 - acc: 0.9994
Epoch 88/100
244/244 [==============================] - 5s 20ms/step - loss: 0.0139 - acc: 0.9997
Epoch 89/100
244/244 [==============================] - 5s 19ms/step - loss: 0.0129 - acc: 0.9997
Epoch 90/100
244/244 [==============================] - 5s 19ms/step - loss: 0.0122 - acc: 0.9999
Epoch 91/100
244/244 [==============================] - 4s 18ms/step - loss: 0.0118 - acc: 0.9997
Epoch 92/100
244/244 [==============================] - 5s 20ms/step - loss: 0.0113 - acc: 0.9997
Epoch 93/100
244/244 [==============================] - 5s 19ms/step - loss: 0.0104 - acc: 0.9999
Epoch 94/100
244/244 [===========

### Prediction and evaluation

In [11]:
dna_ds = DNA_TEST.to_tf_dataset().batch(32).map(tf.sparse.to_dense)
label_ds = ReduceDims(LABELS_TEST)

In [12]:

# convert the prediction to a cover object
pred = model.predict(dna_ds)


In [20]:
print("AUC:", roc_auc_score(label_ds[:], pred))

AUC: 0.9877



### Additional utilities

While janggu.data offers some Wrapper classes (e.g. ReduceDim) that can be used to reshape the original dataset or perform data augmentation during training, some of the functionality can be easily done using the tensorflow.data pipeline directly, including ReduceDims, Squeeze, etc.

However, some specialized methods that can be used for data augmentation are provided by janggu.data.tf_dataset_utils. These include:

`nan_to_zero`: This function can be used if the genomic datasource contains NaNs which one wants to convert to zeros. This can happen in bigwig-file for example.

`random_orientation`: This function randomly flips the strand orientation. It can be used, for example, if a peak signal (e.g. ChIP-seq binding signals are loaded into a Cover object) is used as input, since the peak shapes are often interpreted witout strand specific orientation.

`random_shift`: This function allows to randomly shift the signal tracks by a given shift size up or downstream. It can be used for data-augmentation for the DNA sequence, but also e.g. for high-resolution Cover tracks.